In [ ]:
%%bash
pip -q install pydicom opencv-python scikit-image pyradiomics

wget -q http://www.inf.ufpr.br/lferrari/imagens_ihq.tar.gz && tar -xf imagens_ihq.tar.gz

In [ ]:
import cv2 as cv
import numpy as np
import os
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from joblib import dump, load
import pydicom as dicom
import radiomics
from radiomics import featureextractor
import SimpleITK as sitk
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
def cut_images(input_path, new_width, new_height, output_path=None):
    """
    Cut images into the desired size and save the output images

    Params:
    input_path = path to the original images
    output_path = path to save the cut images
    new_width = width of the cut images
    new_height = height of the cut images

    Return:
    None
    """

    images_data = {}
    classes = []
    patients = []

    n = 0
    # Browse input path
    for class_dir in os.listdir(input_path):
        class_path = os.path.join(input_path, class_dir)

        # If it is a directory
        if os.path.isdir(class_path):

            # Save image id
            image_id = 1

            # Go through images
            for image_file in os.listdir(class_path):
                image_path = os.path.join(class_path, image_file)

                # Save patient id
                patient = image_file.split("_")[0]
                patients.append(patient)

                image = cv.imread(image_path, cv.IMREAD_GRAYSCALE)

                # If image exists
                if image is not None:

                    # Save subimage id
                    sub_id = 1

                    for i in range(0, image.shape[0], new_height):
                        for j in range(0, image.shape[1], new_width):

                            # Cut image into subimage
                            sub_image = image[i:i+new_height, j:j+new_width]

                            # Image name identifier
                            image_name = f"{patient}-img{image_id}-{sub_id}"

                            # Append image and its label to the dictionary
                            # if image_name not in images_data:
                            #     image_name = f"{patient}2-img{image_id}-{sub_id}"
                            #     images_data[image_name] = sub_image
                            # else:
                            images_data[image_name] = sub_image

                            classes.append(int(class_dir))

                            # Write subimage if an output path was given
                            if output_path != None:
                                # Create dir if it doesn't exist
                                os.makedirs(os.path.join(output_path, class_dir), exist_ok=True)
                                # Output file path
                                output_file = f"{image_name}.png"
                                output_file = os.path.join(output_path, class_dir, output_file)
                                # Save subimage
                                cv.imwrite(output_file, sub_image)

                            sub_id += 1
                            n += 1

                image_id += 1

    return images_data, patients, classes

In [ ]:
def divide_folds(image_names, patients, classes, train_percent):
    """
    Divides a dataset into folds for stratified k-fold cross-validation.

    Params:
    images = list of all images
    patients = list with patient id for each image
    classes = list with class for each image

    Return:
    folds = list of tuples, each tuple is one folder
    """
    # Create a list of unique indexes for patients
    unique_patients = list(set(patients))

    #
    n_folds = 5

    patients_per_fold = len(unique_patients)//n_folds
    left = [(len(unique_patients)-i) for i in range(1, (len(unique_patients)%n_folds)+1)]
    # print(left)

    images_classes = dict(zip(images_names, classes))
    assigned_patients = []
    folds = []

    for i in range(n_folds):

        n_patients = 0

        x = []
        y = []

        if i == n_folds-1:
            for j in left:

                patient = patients[-j]

                imgs_patient = [name for name in image_names if patient in name]
                x = x + imgs_patient
                y = y + [images_classes[key] for key in imgs_patient]
                n_patients += 1
                assigned_patients.append(patient)

        for k in range(len(unique_patients)):

            patient = unique_patients[k]

            if n_patients < patients_per_fold and patient not in assigned_patients:
                imgs_patient = [name for name in image_names if patient in name]
                x = x + imgs_patient
                y = y + [images_classes[key] for key in imgs_patient]
                n_patients += 1
                assigned_patients.append(patient)

        folds.append((x, y))

    return folds


In [ ]:
def apply_thresholds(imgs):
    """
    Apply Otsu's and Adaptative thresholds to images

    Params:
    imgs = list of raw images

    Return:
    imgs_otsu = Otsu's thresholded images
    imgs_adapt = Adaptative thresholded images
    """

    for key, value in images_data.items():

        # Otsu's thresholding
        _, th1 = cv.threshold(value, 100, 1, cv.THRESH_BINARY+cv.THRESH_OTSU)

        # Adaptative gaussian thresholding
        #th2 = cv.adaptiveThreshold(img,255,cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY,11,2)
        th3 = cv.adaptiveThreshold(value, 1, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 11, 2)

        images_data[key] = (value, th1, th3)

    return images_data

In [ ]:
# Cut images into 40x30
images_data, patients, classes = cut_images("imagens_ihq_er", 40, 30)
# import itertools
# blu = dict(itertools.islice(images_data.items(),  100))

# # Divide images into folds
images_names = list(images_data.keys())
folds = divide_folds(images_names, patients, classes, 0.7)

# Update images_data, applying Otsu's and adaptive thresholds
images_data = apply_thresholds(images_data)

dump(folds, 'folds.joblib')

Extract features with PyRadiomics

In [ ]:
def run_extractor(images_data, extractor):
    """
    Extract features using sitk and pyradiomics

    Params:
    imgs = raw images
    otsu = masked images with otsu thresholding
    adapt = masked images with adaptative thresholding
    extractor = pyradiomics extractor

    Returns:
    features_otsu = features for otsu mask
    features_adapt = features for adaptative mask
    """

    data_spacing=[1,1,1]
    features_otsu = {}
    features_adapt = {}

    for key, value in images_data.items():

        # Get raw, Otsu's and adaptive images
        img = value[0]
        img_otsu = value[1]
        img_adapt = value[2]

        sitk_img = sitk.GetImageFromArray(img)
        sitk_img.SetSpacing((1, 1, 1))
        sitk_img = sitk.JoinSeries(sitk_img)

        sitk_otsu = sitk.GetImageFromArray(img_otsu)
        sitk_otsu.SetSpacing((1, 1, 1))
        sitk_otsu = sitk.JoinSeries(sitk_otsu)
        sitk_otsu = sitk.Cast(sitk_otsu, sitk.sitkInt32)

        sitk_adapt = sitk.GetImageFromArray(img_adapt)
        sitk_adapt.SetSpacing((1, 1, 1))
        sitk_adapt = sitk.JoinSeries(sitk_adapt)
        sitk_adapt = sitk.Cast(sitk_otsu, sitk.sitkInt32)

        # Extract features and append them to the proper list
        try:
            ft_otsu = extractor.execute(sitk_img, sitk_otsu)
            features_otsu[key] = ft_otsu

            ft_adapt = extractor.execute(sitk_img, sitk_adapt)
            features_adapt[key] = ft_adapt

        except:
            print(f"{key}, ", end="")
            pass

    return features_otsu, features_adapt



In [ ]:
def conditional_append(element, dest):
    """
    Append element to the list destiny, if element is not in destiny

    Params:
    element = an element of any kind
    dest = a destination list

    Returns:
    destiny = list with appended element if the element was not in there
    """
    if element not in dest:
        dest.append(element)

    return dest

def process_features(feats_o, feats_a):
    """
    Process features, in a way that:
    - features that are dictionaries and strings are removed
    - features that are tuples are separated and each element
    of the tuple is considered one feature
    - other types are converted to float

    Params:
    feats_o = list of Otsu's threshold features
    feats_a = list of adaptativa threshold features

    Returns:
    all_feats_o = Otsu's features processed
    all_feats_a = adaptative features processed
    names = feature names processed
    """

    all_feats_o = {}
    all_feats_a = {}
    names = []

    # For each image in one of the features list
    for key in feats_o:

        # Get features for Otsu's and adaptive for this sample
        sample_o = feats_o[key]
        sample_a = feats_a[key]

        values_o = []
        values_a = []

        # For each feature in the list
        for ft in sample_o:

            # Get the feature's value
            value_o = sample_o[ft]
            value_a = sample_a[ft]

            # If the value is str or dict, ignore it
            if type(value_o) == str or type(value_o) == dict:
                continue
            # If it's a tuple
            elif type(value_o) == tuple:
                for e in range(len(value_o)):
                    # Add and index to the feature name
                    conditional_append(f'{ft}_{e}', names)
                    # Append float values to the lists
                    values_o.append(float(value_o[e]))
                    values_a.append(float(value_a[e]))
            # For other data types, just append the name and float values
            else:
                conditional_append(ft, names)
                values_o.append(float(value_o))
                values_a.append(float(value_a))

        # Append processed features to the general list
        all_feats_o[key] = values_o
        all_feats_a[key] = values_a

    return all_feats_o, all_feats_a, names

def extract_features(images_data):
    """
    Process features, in a way that:
    - features that are dictionaries and strings are removed
    - features that are tuples are separated and each element
    of the tuple is considered one feature
    - other types are converted to float
    Get the features' names, with tuple features indexed

    Params:
    folds = list of tuples containing x_train raw and with thresholds

    Returns:
    all_folds_feats = dictionary containing Otsu's features and adaptive
    features for each fold
    names = feature names
    """

    # Create feature extractor
    !wget -c https://raw.githubusercontent.com/AIM-Harvard/pyradiomics/master/examples/exampleSettings/Params.yaml
    params = 'Params.yaml'
    settings = {'label': 1, 'correctMask': True}
    extractor = featureextractor.RadiomicsFeatureExtractor(params, additionalInfo=True, **settings)

    # Extract features from Otsu's and adaptative
    feats_o, feats_a = run_extractor(images_data, extractor)

    # Process features and get feature names
    all_feats_o, all_feats_a, names = process_features(feats_o, feats_a)

    # Save features in the dictionary
    # keys = list(images_data.keys())
    # for i in range(len(images_data)):
    #     key = key[i]
    #     value = images_data[key]
    #     images_data[key] = (value[0], value[1], value[2], all_feats_o[i], all_feats_a[i])

    return all_feats_o, all_feats_a, names

In [ ]:
# import itertools

# lista = list(images_data.keys())
# p = lista[:10000]
# s = lista[10000:20000]
# t = lista[20000:30000]
# q = lista[30000:40000]

# pp = {k:images_data[k] for k in p}
# ss = {k:images_data[k] for k in s}
# tt = {k:images_data[k] for k in t}
# qq = {k:images_data[k] for k in q}

In [ ]:
all_feats_o, all_feats_a, names = extract_features(images_data)

NameError: name 'extract_features' is not defined

In [ ]:
def save_features(all_feats_o, all_feats_a):

    out_o = 'features_o/'
    out_a = 'features_a/'

    os.makedirs(out_o, exist_ok=True)
    os.makedirs(out_a, exist_ok=True)

    for key in all_feats_o:

        ft_o = all_feats_o[key]
        ft_a = all_feats_a[key]

        filename_o = f'{key}_o.txt'
        filename_a = f'{key}_a.txt'

        with open(os.path.join(out_o, filename_o), 'w') as f:
            for elem in ft_o:
                f.write(f'{elem}\n')

        with open(os.path.join(out_a, filename_a), 'w') as f:
            for elem in ft_a:
                f.write(f'{elem}\n')


In [ ]:
save_features(all_feats_o, all_feats_a, names)
with open('ft_names.txt', 'w') as f:
    f.write('\n'.join(names))

In [ ]:
# Download features
!zip -r /content/features_o.zip /content/features_o
!zip -r /content/features_a.zip /content/features_a

from google.colab import files
files.download("/content/features_o.zip")
files.download("/content/features_a.zip")

Read features and folds and prepare for training the model

In [29]:
def read_files(ft_names_path, folds_path):

    from joblib import load

    ft_names = []
    # open file and read the content in a list
    with open(ft_names_path, 'r') as f:
        for line in f:
            # remove linebreak from a current name
            # linebreak is the last character of each line
            x = line[:-1]

            # add current item to the list
            ft_names.append(str(x))

    folds = load(folds_path)

    return ft_names, folds

In [26]:
def read_features(ft_path, data, label):

    features_o = []
    names_o = []
    labels_o = []

    for ft_file in os.listdir(ft_path):

        ft_o = []

        patient = ft_file.split("-")[0]

        for k, image_name in enumerate(data):
            if patient in image_name:
                # open file and read the content in a list
                try:
                    #print(f'achei o {patient} no fold')
                    with open(os.path.join(ft_path, ft_file), 'r') as f:
                        for line in f:
                            # remove linebreak from a current name
                            # linebreak is the last character of each line
                            x = line[:-1]

                            # add current item to the list
                            ft_o.append(float(x))

                    names_o.append(image_name)
                    labels_o.append(label[k])
                except:
                    pass

                break

        if len(ft_o) != 0:
            features_o.append(ft_o)

    return(features_o, names_o, labels_o)

def create_dfs(fold, ft_names):

    x = fold[0]
    y = fold[1]

    ft_o, index_o, label_o = read_features('features_o', x, y)
    df_o = pd.DataFrame(ft_o, columns = ft_names, index = index_o)
    df_o.insert(loc=1, column='label', value=label_o)

    ft_a, index_a, label_a = read_features('features_a', x, y)
    df_a = pd.DataFrame(ft_a, columns = ft_names, index = index_a)
    df_a.insert(loc=1, column='label', value=label_a)

    return df_o, df_a


In [34]:
%%bash
wget https://github.com/vitoriastavis/ufpr-medical-images/raw/main/features_o.tar.gz
wget https://github.com/vitoriastavis/ufpr-medical-images/raw/main/features_a.tar.gz
wget https://github.com/vitoriastavis/ufpr-medical-images/raw/main/folds.tar.gz
wget https://raw.githubusercontent.com/vitoriastavis/ufpr-medical-images/main/ft_names.txt

tar -xf features_o.tar.gz
tar -xf features_a.tar.gz
tar -xf folds.tar.gz

tar: features_a.tar.gz: Not found in archive
tar: Exiting with failure status due to previous errors


In [30]:
# The variable 'folds' is a list of tuples
# folds[0] is the first fold
# in each fold, there are (train_images, test_images, train_classes, test_classes)
# which are used to build the dataframes below
# also using ft_names, which are names of the features to use as column names
ft_names, folds = read_files('ft_names.txt', 'folds.joblib')

In [31]:
df_o, df_a = create_dfs(folds[1], ft_names)

In [33]:
df_a

,diagnostics_Image-original_Spacing_0,diagnostics_Image-original_Spacing_1,diagnostics_Image-original_Spacing_2,diagnostics_Image-original_Size_0,diagnostics_Image-original_Size_1,diagnostics_Image-original_Size_2,diagnostics_Image-original_Mean,diagnostics_Image-original_Minimum,diagnostics_Image-original_Maximum,diagnostics_Mask-original_Spacing_0,...,original_gldm_GrayLevelNonUniformity,original_gldm_GrayLevelVariance,original_gldm_HighGrayLevelEmphasis,original_gldm_LargeDependenceEmphasis,original_gldm_LargeDependenceHighGrayLevelEmphasis,original_gldm_LargeDependenceLowGrayLevelEmphasis,original_gldm_LowGrayLevelEmphasis,original_gldm_SmallDependenceEmphasis,original_gldm_SmallDependenceHighGrayLevelEmphasis,original_gldm_SmallDependenceLowGrayLevelEmphasi
214RE-img77-1,1.0,1.0,1.0,40.0,30.0,1.0,206.462500,106.0,253.0,1.0,...,432.792923,0.309484,7.826999,50.557012,431.137615,6.418623,0.168678,0.057735,0.301310,0.024145
334RE-img50-1,1.0,1.0,1.0,40.0,30.0,1.0,233.355000,177.0,253.0,1.0,...,547.505263,0.179821,1.705263,54.550877,84.098246,47.164035,0.823684,0.030137,0.067562,0.020781
363RE-img1-1,1.0,1.0,1.0,40.0,30.0,1.0,241.134167,187.0,255.0,1.0,...,423.053724,0.241725,2.772894,51.796093,161.290598,24.422466,0.556777,0.033292,0.070171,0.024072
101RE-img3-1,1.0,1.0,1.0,40.0,30.0,1.0,228.627500,190.0,255.0,1.0,...,551.657572,0.123699,1.433834,59.327422,81.019100,53.904502,0.891542,0.022811,0.035552,0.019625
363RE-img1-1,1.0,1.0,1.0,40.0,30.0,1.0,236.930000,169.0,254.0,1.0,...,525.723093,0.231538,5.632184,54.759666,299.283177,11.469262,0.208841,0.030128,0.156346,0.009672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218RE-img14-1,1.0,1.0,1.0,40.0,30.0,1.0,229.998333,52.0,255.0,1.0,...,445.507177,0.526715,12.217225,55.577990,736.887081,4.863922,0.112992,0.038093,0.336949,0.010189
363RE-img1-1,1.0,1.0,1.0,40.0,30.0,1.0,228.870000,133.0,255.0,1.0,...,378.307522,0.450484,6.217920,52.907080,359.293142,11.195028,0.258450,0.037883,0.167773,0.017524
334RE-img50-1,1.0,1.0,1.0,40.0,30.0,1.0,227.544167,185.0,252.0,1.0,...,658.844444,0.042469,1.133333,65.055556,69.197222,64.020139,0.966667,0.020258,0.027427,0.018466
363RE-img1-1,1.0,1.0,1.0,40.0,30.0,1.0,235.652500,163.0,255.0,1.0,...,455.847666,0.219995,3.019656,50.088452,171.083538,19.839681,0.495086,0.032583,0.077917,0.021250


In [ ]:
def knn_classifier(threshold='', data={}):

    ans = {"score" : []}
    print("Classifying...")

    for i in range(5):
        df_o, df_a = create_dfs(folds[i], ft_names)
        if (threshold == 'otsu'):
            df = df_o
        elif (threshold == 'adaptive'):
            df = df_a
        else:
            df = data

        # predictor variables
        X = df.drop(['label'], axis=1).values

        # target variables
        y = df['label'].values

        neigh = KNeighborsClassifier(n_neighbors=3, metric='euclidean')

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

        scaler = StandardScaler()
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        neigh.fit(X_train, y_train)
        score = neigh.score(X_test, y_test)

        print ('Predicting...')
        y_pred = neigh.predict(X_test)

        # show classifier result
        print ('Accuracy: ',  neigh.score(X_test, y_test))

        # creates confusion matrix
        cm = confusion_matrix(y_test, y_pred)
        print (cm)
        print(classification_report(y_test, y_pred))

        ans["score"].append(score)

    print("Classification done!")

    print(ans)
    print (round(np.median(ans["score"])*100, 2))

In [ ]:
def mlp_classifier(threshold='', data={}):
    ans = {"score" : []}
    print("Classifying...")

    for i in range(5):
        df_o, df_a = create_dfs(folds[i], ft_names)
        if (threshold == 'otsu'):
            df = df_o
        elif (threshold == 'adaptive'):
            df = df_a
        else:
            df = data

        # predictor variables
        X = df.drop(['label'], axis=1).values

        # target variables
        y = df['label'].values

        mlp = MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter = 300, activation = 'relu', solver = 'adam')

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

        scaler = StandardScaler()
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        mlp.fit(X_train, y_train)
        score = mlp.score(X_test, y_test)

        print ('Predicting...')
        y_pred = mlp.predict(X_test)

        # show classifier result
        print ('Accuracy: ',  mlp.score(X_test, y_test))

        # creates confusion matrix
        cm = confusion_matrix(y_test, y_pred)
        print (cm)
        print(classification_report(y_test, y_pred))

        ans["score"].append(score)

    print("Classification done!")

    print(ans)
    print (round(np.median(ans["score"])*100, 2))

In [ ]:
knn_classifier('otsu')

In [ ]:
knn_classifier('adaptive')

In [ ]:
mlp_classifier('otsu')

In [ ]:
mlp_classifier('adaptive')